# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time
import numpy as np

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [18]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/single_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [19]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, 2:]

In [20]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

In [21]:
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))

for n in range(0, 13):
    training_rainfall[:, n] = (training_rainfall[:, n] - np.mean(training_rainfall[:, n])) / np.std(training_rainfall[:, n])
    #training_rainfall[:, n] = (training_rainfall[:, n] - np.min(training_rainfall[:, n])) / (np.max(training_rainfall[:, n]) - np.min(training_rainfall[:, n]))

### 1.1 DEPRECATED

This following section uses two separate datasets. Which is not required.

In [22]:
#training_datafile = "E:/31-12-2020/prepared-data/single_train.npy"
#validation_datafile = "E:/31-12-2020/prepared-data/single_valid.npy"
#
#training_rainfallfile = "E:/31-12-2020/prepared-data/expected_train.npy"
#validation_rainfallfile = "E:/31-12-2020/prepared-data/expected_valid.npy"

In [23]:
#training_videos = np.load(training_datafile)
#validation_videos = np.load(validation_datafile)
#
#training_rainfall = np.load(training_rainfallfile)[:, 2:]
#validation_rainfall = np.load(validation_rainfallfile)[:, 2:]

In [24]:
# We need them in X, Y, COLOURS
#training_videos = np.swapaxes(training_videos, 1, 2)
#training_videos = np.swapaxes(training_videos, 2, 3)
#
#validation_videos = np.swapaxes(validation_videos, 1, 2)
#validation_videos = np.swapaxes(validation_videos, 2, 3)

In [25]:
# Scale the data
#training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
#training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))
#
#validation_videos[:, :, :, 0] = (validation_videos[:, :, :, 0] - np.min(validation_videos[:, :, :, 0])) / (np.max(validation_videos[:, :, :, 0]) - np.min(validation_videos[:, :, :, 0]))
#validation_videos[:, :, :, 1] = (validation_videos[:, :, :, 1] - np.min(validation_videos[:, :, :, 1])) / (np.max(validation_videos[:, :, :, 1]) - np.min(validation_videos[:, :, :, 1]))

## 2. Model Definition

Next, a CNN model architecture is defined.

In [26]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(13, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [27]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    history = model.fit(
        xdata,
        ydata,
        batch_size=2,
        validation_split=0.2,
        callbacks=[tensorboard],
        epochs=50
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [28]:
# Flexible parameters
learning_rate = 0.00001

In [29]:
# Run each model multiple times
for i in range(0, 5):
    run_name = "TEST-sd0.05-{}".format(int(time.time())) # "SL_16_16_16__13__22_t-{}".format(int(time.time()))
    tb = TensorBoard(log_dir=".\\logs\\{}".format(run_name))
    model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
    history = train_model(run_name, tb, model, training_videos, training_rainfall)
    print("Final loss: {}".format(history.history["val_loss"][-1]))

Epoch 1/50
110/110 [==============================] - 6s 41ms/step - loss: 2.0445 - val_loss: 1.1683
Epoch 2/50
110/110 [==============================] - 2s 19ms/step - loss: 0.8908 - val_loss: 1.1293
Epoch 3/50
110/110 [==============================] - 2s 18ms/step - loss: 1.0009 - val_loss: 1.1242
Epoch 4/50
110/110 [==============================] - 2s 18ms/step - loss: 1.0173 - val_loss: 1.1219
Epoch 5/50
110/110 [==============================] - 2s 18ms/step - loss: 0.9215 - val_loss: 1.1168
Epoch 6/50
110/110 [==============================] - 2s 18ms/step - loss: 0.9910 - val_loss: 1.1192
Epoch 7/50
110/110 [==============================] - 2s 16ms/step - loss: 0.9043 - val_loss: 1.1125
Epoch 8/50
110/110 [==============================] - 2s 16ms/step - loss: 0.8814 - val_loss: 1.1181
Epoch 9/50
110/110 [==============================] - 2s 18ms/step - loss: 0.9538 - val_loss: 1.1141
Epoch 10/50
110/110 [==============================] - 2s 16ms/step - loss: 0.9904 - val_lo

110/110 [==============================] - 2s 19ms/step - loss: 0.9481 - val_loss: 1.1183
Epoch 31/50
110/110 [==============================] - 2s 18ms/step - loss: 0.8649 - val_loss: 1.1213
Epoch 32/50
110/110 [==============================] - 2s 20ms/step - loss: 0.8775 - val_loss: 1.1183
Epoch 33/50
110/110 [==============================] - 2s 19ms/step - loss: 1.0261 - val_loss: 1.1211
Epoch 34/50
110/110 [==============================] - 2s 19ms/step - loss: 1.0277 - val_loss: 1.1193
Epoch 35/50
110/110 [==============================] - 2s 18ms/step - loss: 0.9464 - val_loss: 1.1185TA: 0s - loss: 0.946
Epoch 36/50
110/110 [==============================] - 2s 19ms/step - loss: 0.9026 - val_loss: 1.1178
Epoch 37/50
110/110 [==============================] - 2s 18ms/step - loss: 0.9387 - val_loss: 1.1194
Epoch 38/50
110/110 [==============================] - 2s 19ms/step - loss: 0.9573 - val_loss: 1.1186
Epoch 39/50
110/110 [==============================] - 2s 18ms/step - loss

110/110 [==============================] - 2s 16ms/step - loss: 1.0314 - val_loss: 1.1197
Epoch 8/50
110/110 [==============================] - 2s 16ms/step - loss: 0.9740 - val_loss: 1.1192
Epoch 9/50
110/110 [==============================] - 2s 16ms/step - loss: 0.8996 - val_loss: 1.1163
Epoch 10/50
110/110 [==============================] - 2s 16ms/step - loss: 0.8788 - val_loss: 1.1169
Epoch 11/50
110/110 [==============================] - 2s 17ms/step - loss: 1.0278 - val_loss: 1.1146
Epoch 12/50
110/110 [==============================] - 2s 17ms/step - loss: 0.9639 - val_loss: 1.1130
Epoch 13/50
110/110 [==============================] - 2s 16ms/step - loss: 1.0183 - val_loss: 1.1160
Epoch 14/50
110/110 [==============================] - 2s 17ms/step - loss: 0.9705 - val_loss: 1.1145
Epoch 15/50
110/110 [==============================] - 2s 17ms/step - loss: 1.1018 - val_loss: 1.1117
Epoch 16/50
110/110 [==============================] - 2s 17ms/step - loss: 0.9569 - val_loss: 1

110/110 [==============================] - 2s 17ms/step - loss: 0.8931 - val_loss: 1.0521
Epoch 37/50
110/110 [==============================] - 2s 17ms/step - loss: 0.9532 - val_loss: 1.0570
Epoch 38/50
110/110 [==============================] - 2s 17ms/step - loss: 0.8684 - val_loss: 1.0526
Epoch 39/50
110/110 [==============================] - 2s 17ms/step - loss: 0.9878 - val_loss: 1.0533
Epoch 40/50
110/110 [==============================] - 2s 16ms/step - loss: 1.0267 - val_loss: 1.0507
Epoch 41/50
110/110 [==============================] - 2s 17ms/step - loss: 1.0284 - val_loss: 1.0703
Epoch 42/50
110/110 [==============================] - 2s 17ms/step - loss: 1.0609 - val_loss: 1.0472
Epoch 43/50
110/110 [==============================] - 2s 17ms/step - loss: 0.9148 - val_loss: 1.0417
Epoch 44/50
110/110 [==============================] - 2s 17ms/step - loss: 0.8646 - val_loss: 1.0425
Epoch 45/50
110/110 [==============================] - 2s 17ms/step - loss: 0.9094 - val_loss: